# Magnetic Field

From magnet.
From finite solenoid

Infinite solenoid

$\nabla \times H = J + \frac{\partial D}{\partial t}$

loop intergal form
$ \oint H \cdot dl = I_{enc}$

$ H = \frac{B}{\mu}$

$ B = \mu_0 (H + M)$

How do we make magnets? We put them in strong magnetic field and anneal. It locks in. Probably uniform magnetization.


Magnetic potential
Long solenoid behaves like monopoles.

It would be cool to predict and measure the force from magnets based on size and material.


A closed integral considering the magnet to be made of little dipoles.
https://en.wikipedia.org/wiki/Magnetic_dipole
The field of a dipole can be derived from 1/r^2 using two charges close together. It can also be considered to be a primitve topic. Also term in multipole expansion.

$dA dl M = dm$ dipole moment.
$A(r) = \mu_0 $ vector potential
magnetic scalar potential $ \frac{m \cdot r}{4 \pi r^3}$ 

Fixed magnetization vs response. Self consistent response too.

If you drill a little hole into a magnet, what do you measure

A cylinder maggnet should have pretty unifrom field in it

Helmholtz coils.




In [ ]:
from collections import namedtuple
def grad(x): return ("grad", x)
grad = namedtuple("grad", "x")
def div(x): return ("div", x)
def curl(x): return ("curl", x)
r = ("r",)
def norm(x): return ("norm", x)
grad(norm(r))
def vadd(x, y): return ("vadd", x, y)



def do_grad(x):
    match x:
        case Dot(x, y):
            return Dot(do_grad(x), y) + Dot(x, do_grad(y))



grad(x=('norm', ('r',)))

In [91]:
from sympy import *
from sympy.vector import CoordSys3D, Del
N = CoordSys3D('N')
v = 3*N.i + 4*N.j + 5*N.k
d = Del()
def v3(name):
    x,y,z = symbols("{name}_x {name}_y {name}_z".format(name=name))
    return x*N.i + y*N.j + z*N.k
m = v3("m")
rvec = N.x*N.i + N.y*N.j + N.z*N.k
rvec
rsym = symbols("r")
def norm(v): return sqrt(v.dot(v))
r = sqrt(rvec.dot(rvec))
rhat = rvec/r
scalar = m.dot(rvec) / r**3
d(scalar).doit().simplify().subs(r, rsym).subs(r**2, rsym**2)

A = m.cross(rvec) / r**3 
d.cross(A).doit().simplify()

a = symbols("a")
avec = a*N.i
#avec = v3("a")
V = 1/norm(rvec+avec) - 1/norm(rvec-avec)
limit(V.diff(a).simplify(),a,0)
series(V,a,0,n=2)


-2*N.x*a/(N.x**2 + N.y**2 + N.z**2)**(3/2) + O(a**2)

Tough to control rewriting. Pattern matching in sympy is too smart.
Could make dumber matcher.
Dumber mul, add


In [106]:
def match1(e, p):
    todo = [(e,p)]
    subst = {}
    while todo:
        e,p = todo.pop()
        if e == p:
            continue
        if isinstance(p, Wild):
            if p in subst:
                if subst[p] != e:
                    return None
            else:
                subst[p] = e
        elif e.func == p.func and len(e.args) == len(p.args):
            todo.extend(zip(e.args, p.args))
        else:
            return None
    return subst

assert match1(x + y, a + b) == {a:x, b:y}
assert match1(x*y, a + b) is None

def replace1(e, lhs, rhs):
    e = e.func(*[replace1(a, lhs, rhs) for a in e.args])
    m = match1(e, lhs)
    if m is not None:
        return rhs.subs(m)
    else:
        return e
    
replace1(x + y, a + b, b)



TypeError: Symbol.__new__() missing 1 required positional argument: 'name'

In [ ]:
# sympy eq search using axioms
# kdrag?
def to_tptp(e):
    if isinstance(e, Wild):
        return str(e).upper()
    return f"{str(e.func)}({",".join([to_tptp(a) for a in e.args])})"
def axiom(lhs,rhs):
    return "cnf({}, axiom, ({} = {})).".format(lhs, to_tptp(lhs), to_tptp(rhs))

def eq_search(e, rules):
    for lhs,rhs in rules:
        m = match1(e, lhs)
        if m is not None:
            return rhs.subs(m)
    return e



In [ ]:
from kdrag.all import *
import kdrag.theories.vec as vec
import kdrag.theories.real as real

m,r = smt.Consts("m r", vec.Vec3)
vec.cross(m,r)
vec.dot(r,r)
vec.norm2(r)
rlen = real.sqrt(vec.norm2(r))
rlen

ScalarField = ArraySort(vec3.Vec3, real.R)
VecField = ArraySort(vec3.Vec3, vec3.Vec3)
kd.notation.add.define([f,g], Lambda([x], f(x) + g(x)))



d = smt.Function("d", ScalarField, VecField)
#kd.axiom(ForAll([v,u], d(dot(v,y)) == dot(d(v),y) + dot(v,d(y))))




In [101]:
d = Function("d", commutative=False)
smul = Function("smul")
a,b,c = Wild("a"), Wild("b"), Wild("c")


rules = [
    (d(a+b), d(a) + d(b)),
    (d(a-b), d(a) - d(b)),
    (d(a*b), a*d(b) + b*d(a)),
    #(d(a/b), (b*d(a) - a*d(b))/(b**2)),
    #(d(1/a), -d(a)/a**2),
    (d(a**2), 2*a*d(a)),
    (d(sqrt(a)), d(a)/(2*sqrt(a))),
    (d(1), 0),
    (d(0), 0),
    (d(d(a)), 0),
    (d(cos(a)), -sin(a)*d(a)),
    (d(sin(a)), cos(a)*d(a)),
    #(d(a**3), 3*a**2*d(a)),
]
x,y = symbols("x y")
d(x**2).replace(d(a**2), 2*a*d(a))
d(x + y).replace(d(a+b), d(a) + d(b))

def dsimp(e):
    for rule in rules:
        e = e.replace(*rule)
    return e
dsimp(d(x**2 + z*y**2 / x))
dsimp(dsimp(dsimp(d(d(x**2 + z*y**2 / x)))))

d(2*x*d(x)) + d(y**2*(d(z_)/x + 4*z_*d(x**(-1/4))/x**(3/4))) + d(2*y*z_*d(y)/x)

In [79]:
from sympy.physics.quantum.operator import DifferentialOperator
f = Function("f")
x * DifferentialOperator(f(x,y,z).diff(x), x) * x

x**2*DifferentialOperator(Derivative(f(x, y, z_), x),x)

# Maxwell Eq
https://en.wikipedia.org/wiki/Maxwell%27s_equations 
displacement current
D and E

```
% diff is exteriori derivative or some kind of geometric derrivative
diff(d) = rho
diff(b) = zero
diff(h) = 
diff(e)

```

https://en.wikipedia.org/wiki/Flux_tube
"Counting" flux lines / tubes. An interesting integration variation. Set flux size. Relax boundaries to get balanced/equal flux.
Or set number of tubes (well, if we know the tube source, kind of same thing.)

Riemann
Riemann stieljes
Lebesgue - find size of level sets.



The charge in a resistive medium. The continuous RC problem


Charge on wire kinks. Is that real?
Fringe field. Dielectric insertion. That's an insane cheat.

Idealizations and paradoxes.
Some tempting idealizations make you want to put this or that out at ininifty and not care. Some conclusions are paradoxical. An effect becomes small as something geometrically becomes large

A magnetic monopole can be made by a long solenoid.


Dimensionality reductions



magnetohydrodynamiucs https://en.wikipedia.org/wiki/Magnetohydrodynamics
There were some kind domains where like the magnetic field gets locked in

eddy currents
satelite loop rotating around earth


"magnets don't do work"
$F = q(E + v \times B)$

$F = Il \times B$
$F = J \times B$ force desnity?

EMF. https://en.wikipedia.org/wiki/Electromotive_force What a confusing concept. "Effective force" $\nabla \times E = -\partial_t B$ means that only a change magnetic field can create E. But battery does it too. The only thing that pushes electrons on some level (?) is electric field. No also entropic forces like gas being in one side of a piston.
Quantum "forces" if that is even the right semantic category to put them in. Quantum effects can be modelled somewhat cl;assicaly if we fit fake fictious terms in our equations of motion.


meep https://meep.readthedocs.io/en/latest/

empy

lenzs law


charge vortex duality
coninuity in 2d is like curl in 2d



In [ ]:
from z3 import *
M = DeclareSort('M')
ScalarField = ArraySort(M, RealSort())
Vec3Field = DeclareSort('Vec3Field')
E,B = Consts('E B', Vec3Field)
rho = Const('rho', ScalarField)

div = Function('div', Vec3Field, ScalarField)
grad = Function('grad', ScalarField, Vec3Field)
curl = Function('curl', Vec3Field, Vec3Field)

vzero = Const('vzero', Vec3Field)
szero = Const("szero", ScalarField)
div(E) == szero

laplacian = Function('laplacian', ScalarField, ScalarField)
hodge = Function('hodge', Vec3Field, Vec3Field)
curl(curl(E)) = grad(div(E)) - laplacian(E)



In [6]:
import sympy as sp
x,y,z,t = sp.symbols("x y z t")
def VecField(name):
    return [sp.Function(name+"{n}")(x,y,z,t) for n in range(3)]
def curl(f):
    return [f[1].diff(z) - f[2].diff(y),
            f[2].diff(x) - f[0].diff(z),
            f[0].diff(y) - f[1].diff(x)]
E = VecField("E")
B = VecField("B")
J = VecField("J")
rho = sp.Function("rho")(x,y,z,t)
def div(f):
    return f[0].diff(x) + f[1].diff(y) + f[2].diff(z)

def cont(rho,J):
    return sp.Eq(rho.diff(t), div(J))

sp.Eq(curl(E), [B.diff(t) for Bx in B])


AttributeError: 'list' object has no attribute 'diff'

In [ ]:
# https://docs.scipy.org/doc/scipy/reference/optimize.html
# integrate some polynomials
# sin, cos, exp, log, etc.
# maybe some 

def tube_integ(f,N):
    h = 1/N
    x = 0
    for i in range(N):
        x += f(i*h)*h
    return x

In [1]:
import meep as mp
cell = mp.Vector3(16,8,0)
geometry = [mp.Block(mp.Vector3(mp.inf,1,mp.inf),
                     center=mp.Vector3(),
                     material=mp.Medium(epsilon=12))]
sources = [mp.Source(mp.ContinuousSource(frequency=0.15),
                     component=mp.Ez,
                     center=mp.Vector3(-7,0))]
pml_layers = [mp.PML(1.0)]
resolution = 10
sim = mp.Simulation(cell_size=cell,
                    boundary_layers=pml_layers,
                    geometry=geometry,
                    sources=sources,
                    resolution=resolution)
sim.run(until=200)
sim.plot2D(fields=mp.Ez)



AttributeError: module 'meep' has no attribute 'Vector3'